# GPT4o code for sentiment labelling (run in colab, downloaded) October Version Final

Here we design a prompt to request sentiment label from GPT4o-mini model by chunks, further adopting cost-effective batch processing  
You will need to setup library, API Key and load the processed sentence level file to conduct query.
It will cost ~USD30 to query the whole IMDA Call Centern Design sessions text 

**WARNING:** As syntax of library, model checkpoint updating along the way,
additional edition of this notebook is necessary to reproduce the results!

# Setup and load data

In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


In [ ]:
import time, json
from openai import OpenAI
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

client = OpenAI(api_key = 'sk-proj-NNn2ogDOcAahK91dPqzAT3BlbkFJoUPvh8YGkkCkzlOtx4sL', timeout=30)

Mounted at /content/drive


In [ ]:
# data_df = pd.read_csv('/content/drive/MyDrive/PLP/input/IMDA_original3_RAW_176K.csv')
data_df = pd.read_csv('/content/drive/MyDrive/PLP/input/IMDA_original1_RAW_180K.csv')
# remove unnamed column
# data_df.drop(columns=['Unnamed: 0'], inplace=True)
# remove none session id
data_df = data_df[~data_df['session_id'].isna()]
# cast session_id into integer
data_df['session_id'] = data_df['session_id'].astype(int)
data_df['speaker_id'] = data_df['speaker_id'].astype(int)
# replace "'" to avoid potention quotation mark in json encoding/decoding issue
data_df['cleaned_text_for_sentiment'] = data_df['cleaned_text_for_sentiment'].str.replace("'"," ")
data_df[['session_id', 'dialog_type', 'x_min', 'x_max', 'cleaned_text_for_sentiment']].head()

,session_id,dialog_type,x_min,x_max,cleaned_text_for_sentiment
0,683,bank,0.00,2.34,(ppo)
1,683,bank,2.34,4.00,okay
2,683,bank,2.48,5.35,hi this is A B C bank how can I help you
3,683,bank,5.35,15.42,hi my name is john (uh) I m calling in with in...
4,683,bank,13.12,14.74,okay (mmhmm)


In [ ]:
data_df.shape[0]/200   # check how many query required given certain chunk size

925.925

# GPT to assign sentiment score

## prompt part & request function definition

In [ ]:
sentiment_system_prompt ='''
You are an experienced Singaporean call center agent tasked with analyzing the sentiment of text from customer service calls.
These calls are inbound inquiries related to banking, insurance, telecom, hotel, holiday, restaurant, as well as government-related queries such as MOE (Singapore Ministry of Education), MSF (Singapore Ministry of Social and Family Development), and HDB (Singapore Housing and Development Board), etc.
Your goal is to rate the sentiment of each text chunk on a scale from -1.00 (negative) to 1.00 (positive) with 2 decimal precision. Additionally, provide a short explanation for each rating, explaining how the score is determined and why it is reasonable.
Customers often use Singlish—a mix of English and local slang—during the conversations, using various slang terms.
Key Singlish Slang and Their Meanings:
Shiok: good
Sian: boring
Jialat: troublesome
Siao: crazy
Paiseh: sorry
Kao Pei: scold
Bojio: never invite
Suay: unfortunate
Pokkai: bankrupt
Atas: high-class
Kena: suffered
Kan Cheong: anxious

Sentiment Rating Scale with Examples
-1.00 (negative):
"Bad experiences that I have so I just want to clarify everything before I leave."
"Yes, that is very bad, so that's the reason why I want to change provider."

-0.70 (negative):
"Recently I've been experiencing some slow Wi-Fi connection, so I would like to ask what is the problem."
"But then it was too late, so the flight left."
"or something cause I can t really travel and all of those are getting wasted now"

-0.50 (mild negative):
"We won't be able to claim this."
"Due to all this incident, I actually missed my flight."
"by the airlines is it or like people stole it"
"if you are (uh) hosp~ hospitalised you have to provide with the (ppb) (uh) the invoice from the hospital and (ppb) (uh) the doctor s (uh) memo to (uh) upon thirty days before"
"like claiming travel insurance recently I went (ppb) for a trip and I lost my (ppb) baggage I think it was stolen so (ppb) (um) I want"
"twelve ya if you ll do at twelve month there will be some penalty incur"
"okay so let s say for those people who under group A they need to pay a higher premium for their personal accident insurance"
"yes correct rebooking fee and it was because of the last minute flight so it was very expensive"

-0.30 (slight negative):
"Okay, but if I exit that time frame, do I still get any coverage?"
"no (uh) (uh) carry on sorry"
"ya late payment charges"
""there was a rebooking fee too is it""
"how much extra would that cost and and if its like does it apply to any countries"
"(uh) engaging in any high risk activities too"
"[orh] so just now the hundred eighty two hundred twenty that [one] is (ppb) for high risk or for low risk"
"so it takes five to six working days so okay can but I will have to make the down payment before I can apply for the car loan right"

-0.15 (neutral):
"Nope, he doesn't have any line now, ya."
"In case I miss the payment date."
"ya sorry"

0.00 (neutral):
"Okay, it follows if I were to travel overseas, or does it only apply?"
"Okay, so annual plan for just yourself or for three of you?"
"yes the price will differ depending on the storage of the phone"

0.15 (neutral):
"Something like Huawei or Oppo would be fine."
"Yeah, I think it would be best if we buy for both."
"ya okay"
"(mm) fine"
"bye bye"
"[ah] I see okay"
"correct"
"sure may I know I m talking to"

0.30 (slight positive):
"Hi, good morning."
"How can I help you?"
"I see, I see. Okay."
"okay okay"
"yes yes"
"okay alright"
"which is more of a bargain"
"okay is there anything else I could help you"

0.50 (mild positive):
"Storage, I think 256GB is good enough already."
"I think the rewards card will suit you the best."
"okay sure can thank you bye"
"and also you are able to enjoy cashback for your land transport for example S_M_R_T buses and taxis"

0.70 (positive):
"Oh, I see, that's great. Okay, thank you."
"Oh, it's good to hear that, yeah, because I want to be unique."
"yes very good condition"
"okay I think I meet the criteria [lah] the eligibility"

1.00 (positive):
"Yes, sure, I will. Thank you very much."
"No, that's all. You've been wonderful, thank you."
"Thank you so much."
"This is really helpful."

Introduction and Examples for INPUT & OUTPUT data structure:
You will receive a list of JSON objects, where each JSON object contains 200 text chunks.
Each chunk includes the following fields:

id: an integer, unique identifier representing the chunk number (from 0 to 199 for the first JSON object, from 200 to 399 for the second JSON object, and so on; each JSON object follows this rule, and you will receive one JSON object each time).
speaker_type: a string, representing the speaker, which can be either "client" or "agent."
dialog_type: a string, representing the type of conversation, such as "bnk" (banking), "tel" (telecom), "ins" (insurance), "moe" (Ministry of Education), etc.
cleaned_text_for_sentiment: the text that needs sentiment analysis.
Your Task:
For each input chunk, generate the corresponding sentiment analysis result. The structure of the output must exactly match the input, and each id must have a corresponding sentiment analysis result. The number of ids and their sequence in the output must be exactly the same as in the input.

Output JSON Format:
For each input chunk, generate a corresponding sentiment analysis result, and always structure the output starting with "result" as the key.

id: (same as the id in the input)
GPT4o: (a sentiment score between -1.00 and 1.00, with two decimal places; negative values indicate negative sentiment, positive values indicate positive sentiment)
explanation: (a short explanation describing how the sentiment score was determined and why it is reasonable, limited to a maximum of 26 tokens)

Example Input:
[
  {
    "id": 1,
    "speaker_type": "client",
    "dialog_type": "tel",
    "cleaned_text_for_sentiment": "Wah, the internet speed today damn slow leh."
  },
  {
    "id": 2,
    "speaker_type": "agent",
    "dialog_type": "bnk",
    "cleaned_text_for_sentiment": "Thank you for your patience, we are solving the issue."
  },
  {
    "id": 3,
    "speaker_type": "client",
    "dialog_type": "ins",
    "cleaned_text_for_sentiment": "I missed my payment, what should I do now?"
  }, ...
]

Example Output:
{"result":[
  {
    "id": 1,
    "GPT4o": -0.70,
    "explanation": "Expresses frustration over slow internet speed."
  },
  {
    "id": 2,
    "GPT4o": 0.50,
    "explanation": "Shows positive reassurance about resolving the issue."
  },
  {
    "id": 3,
    "GPT4o": -0.30,
    "explanation": "Worry over missing a payment, seeking advice."
  }, ...
]}

Note:
The input will contain 200 chunks, and the output must provide corresponding sentiment analysis results for all 200 chunks.
The output must keep exactly the same id numbers and follow the same sequence as in the input.
Now, below is a batch of inputs for sentiment analysis. For each item in the list, return a matching output in the format described.
'''

In [ ]:
def get_session_sentence_sentiment_score_gpt(description):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
          {
              "role": "system",
              "content": sentiment_system_prompt
          },
          {
              "role": "user",
              "content": description
          }
        ],
        temperature=0.2,
        response_format={ "type": "json_object"},
        timeout=120
        )
    return response


## session level all sentences score query

In [ ]:
CHUNK_SIZE = 100  # Reduce chunk size to avoid exceeding response length limit

# Split the DataFrame into chunks of 200 rows each
chunks = np.array_split(data_df, np.ceil(len(data_df) / CHUNK_SIZE))

# warm up
input_dict = chunks[0][["speaker_type","dialog_type","cleaned_text_for_sentiment"]].reset_index(names='id').to_dict('records')
try:
    # get Gemini response
    response = get_session_sentence_sentiment_score_gpt(str(input_dict))
except Exception as e:
    print(f"Failed at warm up: API Request error: {e}")
sentiment_scores_dict = json.loads(response.choices[0].message.content)
print("[DEBUG] http response dict: ", sentiment_scores_dict)
queried_result = pd.DataFrame.from_records(sentiment_scores_dict['result'])
assert set(queried_result.columns.tolist()) == set(['id', 'GPT4o', 'explanation'])
temp = chunks[0].join(queried_result.set_index('id'))
if temp['GPT4o'].isna().any():
    print(f"ALERT: Missing LLM value(s) at chunk warm up")
temp.head()

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


[DEBUG] http response dict:  {'result': [{'id': 0, 'GPT4o': 0.0, 'explanation': 'Neutral inquiry about holiday plans.'}, {'id': 1, 'GPT4o': 0.3, 'explanation': 'Polite greeting and willingness to assist.'}, {'id': 2, 'GPT4o': 0.5, 'explanation': 'Client expresses need for help, indicating positive engagement.'}, {'id': 3, 'GPT4o': 0.3, 'explanation': 'Client shows interest in travel options, slightly positive.'}, {'id': 4, 'GPT4o': 0.5, 'explanation': 'Agent provides options, indicating helpfulness.'}, {'id': 5, 'GPT4o': 0.15, 'explanation': 'Client provides travel dates, neutral response.'}, {'id': 6, 'GPT4o': 0.0, 'explanation': 'Non-verbal response, neutral.'}, {'id': 7, 'GPT4o': 0.3, 'explanation': 'Agent seeks more information, positive engagement.'}, {'id': 8, 'GPT4o': 0.3, 'explanation': 'Client provides family details, positive interaction.'}, {'id': 9, 'GPT4o': 0.0, 'explanation': 'Non-verbal response, neutral.'}, {'id': 10, 'GPT4o': 0.3, 'explanation': 'Agent confirms details

,file_name,session_id,speaker_id,speaker_type,dialog_type,x_min,x_max,text,cleaned_text_for_sentiment,word_count,duration,qualified_for_sentiment,GPT4o,explanation
0,app_0302_3604_phnd_cc-hol.TextGrid,302,3604,client,hol,0.00000,3.09350,call three holiday,call three holiday,3,3.09350,False,0.0,Neutral inquiry about holiday plans.
1,app_0302_0018_phnd_cc-hol.TextGrid,302,18,agent,hol,3.12927,8.50931,hi good afternoon this is lily from A B C trav...,hi good afternoon this is lily from A B C trav...,17,5.38004,True,0.3,Polite greeting and willingness to assist.
2,app_0302_3604_phnd_cc-hol.TextGrid,302,3604,client,hol,8.22110,21.06413,hi (uh) lily (uh) I'm joyce here (ppb) (um) I'...,hi (uh) lily (uh) I m joyce here (ppb) (um) I ...,20,12.84303,True,0.5,"Client expresses need for help, indicating pos..."
3,app_0302_3604_phnd_cc-hol.TextGrid,302,3604,client,hol,21.06413,30.21838,(um) I'm looking into (um) a package to either...,(um) I m looking into (um) a package to either...,15,9.15425,True,0.3,"Client shows interest in travel options, sligh..."
4,app_0302_0018_phnd_cc-hol.TextGrid,302,18,agent,hol,30.81900,42.98125,hi miss joy we do have a package to korea and ...,hi miss joy we do have a package to korea and ...,24,12.16225,True,0.5,"Agent provides options, indicating helpfulness."


## processing main

In [ ]:
failed_records = []
faild_input_dfs = []
queried_result_dfs = []
merged_result_df = pd.DataFrame()

for i, chunk in enumerate(chunks):
    if i > 10:  # for test
        break

    print(f"Processing chunk {i+1}/{len(chunks)}")

    input_dict = chunk[["speaker_type","dialog_type","cleaned_text_for_sentiment"]].reset_index(names='id').to_dict('records')
    try:
        # get Gemini response
        response = get_session_sentence_sentiment_score_gpt(str(input_dict))
    except Exception as e:
      print(f"Failed at chunk {i+1}: API Request error: {e}")
      faild_input_dfs.append(chunk)
      failed_records.append({"id":i+1, "desc": f"API Request error: {e}"})
      queried_result_dfs.append(pd.DataFrame())
      continue

    try:
        if response:
            sentiment_scores_dict = json.loads(response.choices[0].message.content)
            print("[DEBUG] http response dict: ", sentiment_scores_dict)
            queried_result = pd.DataFrame.from_records(sentiment_scores_dict['result'])
            queried_result_dfs.append(queried_result)
        else:
            print(f"Failed at chunk {i+1}: Empty Result!")
            faild_input_dfs.append(chunk)
            failed_records.append({"id":i+1, "desc": f"Empty Result"})
            queried_result_dfs.append(pd.DataFrame())
            continue
    except Exception as e:
      print(f"Failed at chunk {i+1}: JSON decode and to dataframe!")
      faild_input_dfs.append(chunk)
      failed_records.append({"id":i+1, "desc": f"JSON decode and to dataframe: {e}"})
      queried_result_dfs.append(pd.DataFrame())
      continue
    # join result to input chunk
    try:
      assert set(queried_result.columns.tolist()) == set(['id', 'GPT4o', 'explanation'])
      temp = chunk.join(queried_result.set_index('id'))
      # Check for missing GPT4o values
      if temp['GPT4o'].isna().any():
        # Version 1: still add the rest matched result into the dataframe
        print(f"ALERT: Missing LLM value(s) at chunk {i+1}")
        # # Version 2: Raise an exception and treat as fail
        # raise ValueError(f"ALERT: Missing GPT4o value(s) at chunk {i+1}")
      merged_result_df = pd.concat([merged_result_df, temp])
    except Exception as e:
      print(f"Failed at chunk {i+1}: Could not join result: {e}")
      faild_input_dfs.append(chunk)
      failed_records.append({"id":i+1, "desc": f"Could not join result: {e}"})
      continue

    time.sleep(5)
print(merged_result_df.shape)
print(failed_records)

Processing chunk 1/1763
[DEBUG] http response dict:  {'result': [{'id': 0, 'GPT4o': 0.0, 'explanation': 'Neutral inquiry about holiday, no positive or negative sentiment.'}, {'id': 1, 'GPT4o': 0.3, 'explanation': 'Agent greets warmly, indicating a slight positive tone.'}, {'id': 2, 'GPT4o': 0.5, 'explanation': 'Client expresses need for help, showing mild positivity.'}, {'id': 3, 'GPT4o': 0.3, 'explanation': 'Client is looking for recommendations, indicating slight positivity.'}, {'id': 4, 'GPT4o': 0.3, 'explanation': 'Agent provides information, maintaining a positive tone.'}, {'id': 5, 'GPT4o': 0.0, 'explanation': 'Client states travel dates neutrally, no sentiment expressed.'}, {'id': 6, 'GPT4o': 0.0, 'explanation': "Client's response is neutral, no sentiment conveyed."}, {'id': 7, 'GPT4o': 0.3, 'explanation': 'Agent seeks more information, maintaining a positive engagement.'}, {'id': 8, 'GPT4o': 0.3, 'explanation': 'Client provides details positively about group size.'}, {'id': 9, 

### score distribution checks

In [ ]:
merged_result_df.GPT4o.value_counts()

,count
GPT4o,
0.00,66243
0.30,38736
0.50,27927
0.70,8674
0.15,8589
-0.30,5015
-0.15,1853
-0.50,1607
-0.70,527


In [ ]:
print("number of records Null Value matched: ", merged_result_df[merged_result_df['GPT4o'].isna()].shape[0])
# merged_result_df[merged_result_df['GPT4o'].isna()]
merged_result_df.to_excel("/content/drive/MyDrive/PLP/IMDA_Original/IMDA_original3_V3b_Gemini_0906_gpt4o.xlsx",index=False)
merged_result_df1 = merged_result_df[~merged_result_df['GPT4o'].isna()]
merged_result_df1.to_excel("/content/drive/MyDrive/PLP/IMDA_Original/IMDA_original3_V3b_Gemini_0906_gpt4o_ra.xlsx",index=False)
merged_result_df1.shape

number of records Null Value matched:  0


(4900, 14)

In [ ]:
print("number of failure: ", len(faild_input_dfs))
pd.DataFrame.from_records(failed_records).to_csv(
    "/content/drive/MyDrive/PLP/IMDA_Original/Fail_records_IMDA_original3_V3_Gemini_0906_failed_reason.csv",index=False)

failed_input_concat_df = pd.DataFrame()
for failed_input_df in faild_input_dfs:
    failed_input_concat_df = pd.concat([failed_input_concat_df,failed_input_df])
failed_input_concat_df.to_excel("/content/drive/MyDrive/PLP/IMDA_Original/IMDA_original3_V3_Gemini_0906_failed_input.xlsx",index=False)
failed_input_concat_df.shape[0], failed_input_concat_df.shape[0]//100

# Batch API
Ref: https://platform.openai.com/docs/guides/batch/getting-started

In [ ]:
# len(chunks)
batch_id_list = list(range(100, 501, 100))
batch_file_inputs = []
for batch_id_end in batch_id_list:
    batch_file = "/content/drive/MyDrive/PLP/IMDA_Original/batch_requests_session2_" + str(batch_id_end) + ".jsonl"
    batch_file_inputs.append(batch_file)
print(batch_file_inputs)

['/content/drive/MyDrive/PLP/IMDA_Original/batch_requests_session2_100.jsonl', '/content/drive/MyDrive/PLP/IMDA_Original/batch_requests_session2_200.jsonl', '/content/drive/MyDrive/PLP/IMDA_Original/batch_requests_session2_300.jsonl', '/content/drive/MyDrive/PLP/IMDA_Original/batch_requests_session2_400.jsonl', '/content/drive/MyDrive/PLP/IMDA_Original/batch_requests_session2_500.jsonl']


In [ ]:
CHUNK_SIZE = 200  # Reduce chunk size to avoid exceeding response length limit
CHUNK_PER_BATCH = 100
# Split the DataFrame into chunks of 200 rows each
chunks = np.array_split(data_df, np.ceil(len(data_df) / CHUNK_SIZE))

batch_file_inputs = []
for batch_id_end in batch_id_list:
    batch_file = "/content/drive/MyDrive/PLP/IMDA_Original/batch_requests_session2_" + str(batch_id_end) + ".jsonl"
    batch_file_inputs.append(batch_file)
    with open(batch_file, 'w') as file:
        # if  batch_id_end > len(chunks):
        #     batch_id_end = len(chunks)
        for i in range(batch_id_end-CHUNK_PER_BATCH, batch_id_end):
            chunk = chunks[i]
            # print(f"Processing chunk_id: {i}")
            input_dict = chunk[["speaker_type","dialog_type","cleaned_text_for_sentiment"]].reset_index(names='id').to_dict('records')
            # input_dict = dialog_df["cleaned_text_for_sentiment"].to_dict()
            request = {
                "custom_id": f"request-session2-batch-{batch_id_end//CHUNK_PER_BATCH}-chunk-{i}",
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                    "model":"gpt-4o-mini",
                    "messages": [
                        {
                            "role": "system",
                            "content": sentiment_system_prompt
                        },
                        {
                            "role": "user",
                            "content": str(input_dict)
                        }
                    ],
                    "temperature": 0.2,
                    "response_format": {"type": "json_object"}
                }
            }
            file.write(json.dumps(request) + "\n")
print(batch_file_inputs)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


['/content/drive/MyDrive/PLP/IMDA_Original/batch_requests_session2_100.jsonl', '/content/drive/MyDrive/PLP/IMDA_Original/batch_requests_session2_200.jsonl', '/content/drive/MyDrive/PLP/IMDA_Original/batch_requests_session2_300.jsonl', '/content/drive/MyDrive/PLP/IMDA_Original/batch_requests_session2_400.jsonl', '/content/drive/MyDrive/PLP/IMDA_Original/batch_requests_session2_500.jsonl']


In [ ]:
batch_input_file = client.files.create(
  file=open('/content/drive/MyDrive/PLP/IMDA_Original/batch_requests_session2_500.jsonl', "rb"),
  purpose="batch"
)

batch_input_file_id = batch_input_file.id

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "batch API session2 400-500 chunks"
    }
)

Batch(id='batch_JFgr8DILPOWXeKvqLjfQMSci', completion_window='24h', created_at=1726064154, endpoint='/v1/chat/completions', input_file_id='file-doKC2hdk1IbeYxV4PzL3GAv3', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1726150554, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'batch API session2 400-500 chunks'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [ ]:
client.batches.retrieve("batch_JFgr8DILPOWXeKvqLjfQMSci")

Batch(id='batch_JFgr8DILPOWXeKvqLjfQMSci', completion_window='24h', created_at=1726064154, endpoint='/v1/chat/completions', input_file_id='file-doKC2hdk1IbeYxV4PzL3GAv3', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1726071920, error_file_id=None, errors=None, expired_at=None, expires_at=1726150554, failed_at=None, finalizing_at=1726071903, in_progress_at=1726064156, metadata={'description': 'batch API session2 400-500 chunks'}, output_file_id='file-uNfQ5tavAIwRtnJX0QpA8N9u', request_counts=BatchRequestCounts(completed=100, failed=0, total=100))

In [ ]:
# from datetime import datetime
# expires_at = 1725885562
# # Convert to human-readable datetime format
# expires_at_datetime = datetime.utcfromtimestamp(expires_at)
# print("Expires At:", expires_at_datetime)

Expires At: 2024-09-09 12:39:22
Completed At: 2024-09-08 14:34:56


## decode results

In [ ]:
batch_job_id_list_s3 = ["batch_yEjsXaQB2bfRgK3LhsytHCHI", "batch_USEemtKWjPWGw5KDG8CH76UB", # 50,200
                     "batch_rmzSY2IuDVueLFhXAPFyPg8Q", # -350
                     "batch_Cvb19tByVo2Bs9RD19JePNBS", # -500
                     "batch_kJOwmEyWzFtHBwT5dksmlw0e", # -650
                     "batch_MAcB3MtRLU5yybhmEqcJGA8A", # -800
                     "batch_bbMnM97zNYqehbpZjUan3fyQ"
]
batch_job_id_list_s1 = ['batch_mCrX5jAWzpHSZ7gDP8etzrIR','batch_sM522HVdXBCkQxICkE9sDnwt',
                        'batch_66zWtY3XgLq7sgLfsIJPFqYZ','batch_emuv4islvInxAUXxM2DRKoBa',
                        'batch_wU1ck47OxitAUPQVmX91s47e',
                        'batch_QOZ1jeIcTN7dCXM0wFKRqOXp', # 750-900
                        'batch_M8lcc3nynKFjQw6YuSln8je0', # 900-1050
                        'batch_LWyISyNfd4eKWA4EicyFnwCe', # 1050-1200
                        'batch_6sn5W6UlGOq3Pz4SXqosZUGh'
                        ]

batch_job_id_list_s2 = ['batch_3XhCo215015KutXEhEoifEY9','batch_F7RLyLBbaDiLCNpp8GCpDiie',
                        'batch_kgpCEYg8MOLd2wOe86Z7CBpB','batch_uMya217ooMYBzmjBwXmvHiNe',
                        'batch_JFgr8DILPOWXeKvqLjfQMSci'
                        ]
for job_id in batch_job_id_list_s2:
    print(client.batches.retrieve(job_id))
# client.batches.list(limit=6)

Batch(id='batch_3XhCo215015KutXEhEoifEY9', completion_window='24h', created_at=1725983690, endpoint='/v1/chat/completions', input_file_id='file-mTvw7l9jBkYr8F9ejsLgJ1tC', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1725984159, error_file_id=None, errors=None, expired_at=None, expires_at=1726070090, failed_at=None, finalizing_at=1725984145, in_progress_at=1725983696, metadata={'description': 'batch API session2 0-100 chunks'}, output_file_id='file-KmY7igszF14x2MFqpnhY3gE9', request_counts=BatchRequestCounts(completed=100, failed=0, total=100))
Batch(id='batch_F7RLyLBbaDiLCNpp8GCpDiie', completion_window='24h', created_at=1725984296, endpoint='/v1/chat/completions', input_file_id='file-Iyibl3UkFHs2vzaCKFOcoLrJ', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1725985454, error_file_id=None, errors=None, expired_at=None, expires_at=1726070696, failed_at=None, finalizing_at=1725985438, in_progress_at=172598

In [ ]:
result_files_list_s3 = ["file-ua6U97YXjje6pzKbr4aLNK9v", "file-qLV77h23XMibKlmfCii01FEQ" ,
                     'file-z0ke6DeH6pgJ08v8egZR5w32','file-jywdbJTaZbEmuncloDqXMC37','file-5hzvAdvXd6PJ2WC3xvTsQ5EQ',
                     'file-YDxmXbuUekMiUiAbQOx1zmHw','file-jG6cq5bYw04c7RuzdGdOYEMF']
result_files_list_s1 = ['file-H3LrCVa8lYLHYYCrSvWBccwW', 'file-KkdTMi6qy1wjDbTsUBURjk4L',
                        'file-y1asJimONwMLZ4Wcv9gHJim2','file-YVGxpr6krhp3OPFCggvKf4yA',
                        'file-ncBJjXQjHH85Hc62nYKp2ky6',
                        'file-Bei5XuHeR2kyfnkts5Cfd0wX', 'file-9OZp5JlHC2fQP9Qi29uQpSRb',
                        'file-rT68IuP9KInHWqDdbN1cvmao', 'file-FTSwG1TwMpROL4lIgyitSVpN'
                        ]
result_files_list_s2 = ['file-KmY7igszF14x2MFqpnhY3gE9','file-CJxM5bemGjs6Hyia18kLQgov',
                        'file-QtRjiIlr5HrOeBHyumCwaXnU','file-rQy5PNGgV2udfx91gBPj79CD',
                        'file-uNfQ5tavAIwRtnJX0QpA8N9u'
                        ]
batch_response_list = []
for file_id in result_files_list_s2:
    file_response = client.files.content(file_id)
    batch_response_list += file_response.text.strip().split("\n")
len(batch_response_list)

500

In [ ]:
# file_response = client.files.content("file-ua6U97YXjje6pzKbr4aLNK9v")
# print(file_response.text)

In [ ]:
failed_records = []
faild_input_dfs = []
queried_result_dfs = []
merged_result_df = pd.DataFrame()

for i, line in enumerate(batch_response_list):
    # print(f"Processing chunk {i+1}/{len(chunks)}")
    response = json.loads(line)
    try:
        if response:
            sentiment_result = response['response']['body']['choices'][0]['message']['content']
            sentiment_scores_dict = json.loads(sentiment_result)
            # print("[DEBUG] http response dict: ", sentiment_scores_dict)
            queried_result = pd.DataFrame.from_records(sentiment_scores_dict['result'])
            # if 'eximent' in	queried_result.columns or 'eximentation' in	queried_result.columns:
            #     print(f"ALERT: extra attribute at chunk {i+1} - having eximent/eximentation")
            assert 'GPT4o' in queried_result.columns.tolist()
            assert 'id' in queried_result.columns.tolist()
            # assert set(queried_result.columns.tolist()) == set(['id', 'GPT4o', 'explanation'])
            queried_result_dfs.append(queried_result)
        else:
            print(f"Failed at batch {i+1}: wrong result!")
            print(line[:100])
            failed_records.append({"id":i+1, "desc": f"Empty Result"})
            queried_result_dfs.append(pd.DataFrame())
            continue
    except Exception as e:
      print(f"Failed at chunk {i+1}: JSON decode and to dataframe!")
      failed_records.append({"id":i+1, "desc": f"JSON decode and to dataframe: {e}"})
      queried_result_dfs.append(pd.DataFrame())
      continue

In [ ]:
queried_result_concat = pd.concat(queried_result_dfs).dropna(subset=['id'])
queried_result_concat = queried_result_concat[['id','GPT4o','explanation']]
queried_result_concat['id_numeric'] = pd.to_numeric(queried_result_concat['id'], errors='coerce')
queried_result_concat = queried_result_concat.dropna()
queried_result_concat['id'] = queried_result_concat['id_numeric']
queried_result_concat = queried_result_concat.drop(columns = ['id_numeric'])
queried_result_concat = queried_result_concat.drop_duplicates(subset=['id'])

queried_result_concat

,id,GPT4o,explanation
0,0,0.00,"Neutral text, no sentiment expressed."
1,1,0.00,"Neutral response, lacks emotional content."
2,2,0.30,"Polite greeting, indicates willingness to assist."
3,3,0.30,"Client is polite and inquiring, showing interest."
4,4,0.15,"Agent acknowledges client, slight positive tone."
...,...,...,...
195,99995,0.50,Inquires about assistance positively.
196,99996,0.50,Expresses willingness to help with questions p...
197,99997,0.30,"Seeks clarification on client identity, indica..."
198,99998,0.50,Confirms client’s details positively.


In [ ]:
merged_result_df = data_df.join(queried_result_concat.set_index('id'))
merged_result_df

,file_name,session_id,speaker_id,speaker_type,dialog_type,x_min,x_max,text,cleaned_text_for_sentiment,word_count,duration,qualified_for_sentiment,GPT4o,explanation
0,app_0683_4366_phnd_cc-bnk.TextGrid,683,4366,client,bank,0.00000,2.34000,(ppo),(ppo),0,2.34000,False,0.00,"Neutral text, no sentiment expressed."
1,app_0683_4366_phnd_cc-bnk.TextGrid,683,4366,client,bank,2.34000,4.00000,okay,okay,1,1.66000,False,0.00,"Neutral response, lacks emotional content."
2,app_0683_0013_phnd_cc-bnk.TextGrid,683,13,agent,bank,2.48000,5.35000,hi this is A B C bank how can I help you,hi this is A B C bank how can I help you,12,2.87000,True,0.30,"Polite greeting, indicates willingness to assist."
3,app_0683_4366_phnd_cc-bnk.TextGrid,683,4366,client,bank,5.35000,15.42000,hi my name is john (uh) I'm calling in with in...,hi my name is john (uh) I m calling in with in...,20,10.07000,True,0.30,"Client is polite and inquiring, showing interest."
4,app_0683_0013_phnd_cc-bnk.TextGrid,683,13,agent,bank,13.12000,14.74000,okay (mmhmm),okay (mmhmm),1,1.62000,False,0.15,"Agent acknowledges client, slight positive tone."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185180,app_1355_5710_phnd_cc-tel.TextGrid,1355,5710,client,telecom,600.07556,612.60436,(mm),(mm),0,12.52880,False,NaN,NaN
185181,app_1355_5710_phnd_cc-tel.TextGrid,1355,5710,client,telecom,612.60436,618.09106,(oo) okay I see thank you so much lily yup I'l...,(oo) okay I see thank you so much lily yup I l...,15,5.48670,True,NaN,NaN
185182,app_1355_0018_phnd_cc-tel.TextGrid,1355,18,agent,telecom,618.20844,624.64712,sure thank you so much for your call if you ne...,sure thank you so much for your call if you ne...,19,6.43868,True,NaN,NaN
185183,app_1355_0018_phnd_cc-tel.TextGrid,1355,18,agent,telecom,624.64712,629.61963,thank you have a nice day bye bye,thank you have a nice day bye bye,7,4.97251,True,NaN,NaN


In [ ]:
print("number of records Null Value matched: ", merged_result_df[merged_result_df['GPT4o'].isna()].shape[0])
# merged_result_df[merged_result_df['GPT4o'].isna()]
merged_result_df.to_excel('/content/drive/MyDrive/PLP/IMDA_Output/IMDA_sesson2_V3b_gpt_0912.xlsx',index=False)
merged_result_df1 = merged_result_df[~merged_result_df['GPT4o'].isna()]
merged_result_df1.to_excel('/content/drive/MyDrive/PLP/IMDA_Output/IMDA_sesson2_V3b_gpt_0912_ra.xlsx',index=False)
merged_result_df1.shape

number of records Null Value matched:  85624


(99561, 14)

In [ ]:
merged_result_df1.shape

(258828, 14)